In [1]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torch

from main_code.skin_segmentation_dataset import SkinSegmentationData, label_img_to_rgb
from main_code.skin_segmentation_nn import SkinSegmentationNN
from main_code.skin_segmentation_evaluation import evaluate_model
from main_code.skin_segmentation_visualization import visualizer

from utils.save_load_model import save_model, load_model

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
batch_size = 15

In [ ]:
#Training Dataset and Dataloader
train_data = SkinSegmentationData('dataset/train.txt')
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [2]:
#Validation Dataset and Dataloader
validation_data = SkinSegmentationData('dataset/validate.txt')
validation_dataloader = torch.utils.data.DataLoader(validation_data, batch_size=15, shuffle=False)

In [ ]:
validation_data.image_names[9]

In [ ]:
#train_data.image_names[86]

In [ ]:
num_batches = len(train_dataloader)
num_batches

In [ ]:
#Model Initialization
hparams = {
    'batch_size': batch_size,
    'num_batches': num_batches,
    'num_resnet_trainable':0,
    'optimizer': 'Adam',
    'learning_rate': 0.01,
    'momentum': 0.0
}
model = SkinSegmentationNN(hparams = hparams)

In [ ]:
inputs, targets = iter(train_dataloader).next()

In [ ]:
inputs.shape

In [ ]:
model(inputs).shape

In [ ]:
#Initialize Tensorboard Logger
logger = pl.loggers.TensorBoardLogger(save_dir='lightning_logs', name="logs")

In [ ]:
#Initialize Model Trainer 
trainer = pl.Trainer(
    max_epochs=100,
    logger=logger,
    log_every_n_steps=1,
    callbacks=[EarlyStopping(monitor='val_loss', patience = 10, mode='min')],
    gpus=None,
)

In [ ]:
#Train Model
trainer.fit(model, train_dataloaders = train_dataloader, val_dataloaders = validation_dataloader)

In [ ]:
save_model(model, 'model')

In [ ]:
len(train_data)

In [ ]:
evaluate_model(model, train_dataloader) #latest best for max num train images: 99.07% for 120 images

In [ ]:
visualizer(model, train_data, num_imgs = 5)

In [6]:
len(validation_data), len(train_data)

15

In [ ]:
evaluate_model(model, validation_dataloader) #latest best for max num train images: 64.76% for 16 val images, 120 train images

In [ ]:
visualizer(model, validation_data, num_imgs = 10)

In [3]:
model_loaded = load_model(SkinSegmentationNN, file_name='model')

In [4]:
#Validation Dataset and Dataloader
validation_data = SkinSegmentationData('dataset/validate.txt')
validation_dataloader = torch.utils.data.DataLoader(validation_data, batch_size=15, shuffle=False)
len(validation_data)

In [5]:
evaluate_model(model_loaded, validation_dataloader) #latest best: 68.8% for 15 val images

{'accuracy': 0.6882957637652434,
 'non-skin num instances': 18936,
 'non-skin recall': 0.8259400084495142,
 'non-skin precision': 0.7656533020022519,
 'non-skin f1 score': 0.7946548789472346,
 'skin num instances': 28074,
 'skin recall': 0.5830305620859157,
 'skin precision': 0.7341885709159415,
 'skin f1 score': 0.6499364675984752,
 'acne num instances': 12500,
 'acne recall': 0.71752,
 'acne precision': 0.5334562540890977,
 'acne f1 score': 0.6119469177498037}